In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your data
data = pd.read_csv("./aos_data/merged_phuc_data.csv")
features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ', 'IR']
X = data[features]
y = data['Label'].map({'sleep': 0, 'awake': 1})

# Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train SVM (linear kernel for simplicity)
model = SVC(kernel='linear', C=1.0)  # C controls regularization, adjust if needed
model.fit(X_train, y_train)

# Evaluate
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")
print(f"Number of support vectors: {model.n_support_}")

# Export min/max values
with open("scaler_values.h", "w") as f:
    f.write("// Scaler values\n")
    for i, feature in enumerate(features):
        f.write(f"const float {feature}_min = {scaler.data_min_[i]}f;\n")
        f.write(f"const float {feature}_max = {scaler.data_max_[i]}f;\n")

# Export SVM model to C header
with open("svm_model.h", "w") as f:
    f.write("#ifndef SVM_MODEL_H\n")
    f.write("#define SVM_MODEL_H\n\n")
    f.write(f"const int n_features = {len(features)};\n")
    f.write(f"const int n_support_vectors = {sum(model.n_support_)};\n\n")
    
    # Support vectors
    sv_flat = model.support_vectors_.flatten()
    f.write("const float support_vectors[] = {\n")
    for i, val in enumerate(sv_flat):
        f.write(f"{val}f")
        if i < len(sv_flat) - 1:
            f.write(", ")
        if (i + 1) % 7 == 0:  # Line break every 7 values (n_features)
            f.write("\n")
    f.write("\n};\n\n")
    
    # Dual coefficients (alpha * y)
    coef_flat = model.dual_coef_.flatten()
    f.write("const float dual_coef[] = {\n")
    for i, val in enumerate(coef_flat):
        f.write(f"{val}f")
        if i < len(coef_flat) - 1:
            f.write(", ")
        if (i + 1) % 10 == 0:  # Line break every 10 values
            f.write("\n")
    f.write("\n};\n\n")
    
    # Intercept
    f.write(f"const float intercept = {model.intercept_[0]}f;\n")
    f.write("#endif\n")

print("Model exported to svm_model.h and scaler_values.h")

Accuracy: 1.00
Number of support vectors: [41 44]
Model exported to svm_model.h and scaler_values.h
